<a href="https://colab.research.google.com/github/DDiekmann/Applied-Verification-Lab-Neural-Networks/blob/main/Tutorials/Verification_by_abstraction_with_eran.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial for Neural Network Verification with ERAN

---

*As an example we try to verifiy the **robustness** of a classification Network trained on the MNIST dataset.
We are using [Eran](https://github.com/eth-sri/eran).*

Important Links:
- [Github](https://github.com/eth-sri/eran) 
- [Manual](https://files.sri.inf.ethz.ch/eran/docs/eran_manual.pdf)

---

The next block takes around 6 minutes.

In [ ]:
%%capture
%%bash

git clone https://github.com/eth-sri/ERAN.git

sudo apt install m4
sudo apt install build-essential
sudo apt install autoconf
sudo apt install libtool
sudo apt install texlive-latex-base

cd ERAN
sudo bash ./install.sh
pip install -r requirements.txt

In [ ]:
%pip uninstall -y pillow
%pip install pillow

In [ ]:
%%capture

%pip install onnx onnxruntime

import torch
from torch import nn
import numpy as np
import torch.onnx
import sys

import matplotlib.pyplot as plt

!wget --no-cache --backups=1 {'https://raw.githubusercontent.com/DDiekmann/Applied-Verification-Lab-Neural-Networks/main/lib/mnist_trainer.py'}

import mnist_trainer

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim, number_of_neurons):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_dim, number_of_neurons),
            nn.ReLU(),
            nn.Linear(number_of_neurons, number_of_neurons),
            nn.ReLU(),
            nn.Linear(number_of_neurons, number_of_neurons),
            nn.ReLU(),
            nn.Linear(number_of_neurons, output_dim),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
%%capture
train_dataloader, test_dataloader = mnist_trainer.load_mnist_dataset(batch_size=64)

In [ ]:
model = mnist_trainer.train_model(
    NeuralNetwork(input_dim=28*28, output_dim=10, number_of_neurons=40), 
    epochs=5, 
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    )

In [ ]:
model_filename = "mnist_net.onnx"
dummy_input=torch.randn(1, 28, 28)

# set model to eval mode
model.eval()

# create a dummy input in the shape of the input values
device = "cuda" if torch.cuda.is_available() else "cpu"
dummy_input = dummy_input.to(device)

torch.onnx.export(model,
                  dummy_input,
                  model_filename,
                  export_params=True,
                  verbose=False,
                  input_names=['image'],
                  output_names=['classification'],
                  )

In [ ]:
%%bash

cd ERAN/tf_verify/

python3 . --netname ../../mnist_net.onnx --epsilon 0.1 --domain deepzono --dataset mnist

In [ ]:
%%bash

cd ERAN/tf_verify/

python3 . --netname ../../mnist_net.onnx --epsilon 0.03 --domain deepzono --dataset mnist

In [ ]:
%%bash

cd ERAN/tf_verify/

python3 . --netname ../../mnist_net.onnx --epsilon 0.03 --domain deepzono --from_test 240 --dataset mnist

In [ ]:
%%bash
wget --no-cache --backups=1 {'https://github.com/eth-sri/eran/files/3653882/zonotope_example.txt'}

cd ERAN/tf_verify/

python3 . --netname ../../mnist_net.onnx --zonotope ../../zonotope_example.txt --domain deepzono --dataset mnist

In [ ]:
%%bash

cd ERAN/tf_verify/
python3 . --netname ../../mnist_net.onnx --dataset mnist --domain deeppoly --spatial --t-norm inf --delta 3

In [ ]:
%%bash

#wget --no-cache --backups=1 {'https://files.sri.inf.ethz.ch/eran/nets/onnx/mnist/mnist_relu_3_50.onnx'}

cd ERAN/tf_verify/

python3 . --netname ../../mnist_relu_3_50.onnx --epsilon 0.1 --domain deepzono --dataset mnist

In [ ]:
%%bash

cd ERAN/tf_verify/

python3 . --netname ../../ffnnRELU__PGDK_w_0_3_6_500.onnx --epsilon 0.1 --domain deepzono --dataset mnist

In [ ]:
!wget --no-cache --backups=1 {'https://raw.githubusercontent.com/eth-sri/deepg/master/code/examples/example1/config.txt'}

In [ ]:
%%bash

cd ERAN

set -e

cd gurobi912/linux64/src/build
sed -ie 's/^C++FLAGS =.*$/& -fPIC/' Makefile
make
cp libgurobi_c++.a ../../lib/
cp ../../lib/libgurobi90.so /usr/lib
cd ..

export GUROBI_HOME="$(pwd)/gurobi900/linux64"
export PATH="${PATH}:${GUROBI_HOME}/bin"
export CPATH="${CPATH}:${GUROBI_HOME}/include"
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/lib:${GUROBI_HOME}/lib

git clone https://github.com/eth-sri/deepg.git
cd deepg/code
mkdir build
make shared_object
cp ./build/libgeometric.so /usr/lib
cd ../..

export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/lib:/usr/lib

ldconfig

Footer

cd tf_verify/

python3 . --netname ../../mnist_net.onnx --geometric --geometric_config ../../config.txt --num_params 1 --dataset mnist